In [1]:
import requests
import urllib
import json

In [9]:
class FortiAPIClient():
    
    def __init__(self, host, session=None, is_https=False):
        self._session = session or requests.session()
        self._is_https = is_https
        self._host = host
        
    def get_url(self, path):
        if self._is_https:
            prefix = f'https://{self._host}'
        else:
            prefix = f'http://{self._host}'
        url = f'{prefix}/{path}'
        return url
        
    def login(self, username, password):
        url = self.get_url('logincheck')
        username = username
        password = password
        data = f'username={urllib.parse.quote(username)}&secretkey={urllib.parse.quote(password)}'
        response = self._session.post(url=url, data=data)
        return response

In [10]:
fac = FortiAPIClient('10.210.201.5')

In [11]:
fac.login(username='admin', password='forti666')

<Response [200]>

In [ ]:
# To be wrapped
# To be wrapped
# To be wrapped
# To be wrapped

In [ ]:
# Login
session = requests.session()
url = 'http://10.210.201.5/logincheck'
username = 'admin'
password = 'forti666'
data = f'username={urllib.parse.quote(username)}&secretkey={urllib.parse.quote(password)}'
response = session.post(url=url, data=data)

In [ ]:
# Login with token (to be fixed)
# https://kb.fortinet.com/kb/documentLink.do?externalID=FD45595
# https://github.com/fortinet-solutions-cse/fortiosapi/blob/f3ac6561de5588c600a7ade8ff6c742c81b8dfcf/fortiosapi/fortiosapi.py#L231
session = requests.session()
access_token = '675QrhHwst0fts4gbfhQGrHjwfrH63'
url = f'http://10.210.201.5/api/v2/cmdb/firewall/address'
headers = {'Authorization': 'Bearer ' + access_token}
session.headers.update(headers)
response = session.get(url=url)

In [ ]:
# Request - get (format=name|subnet)
url = 'http://10.210.201.5/api/v2/cmdb/firewall/address?format=name|subnet'
response = session.get(url=url)
print(response.text)

In [ ]:
# Request - get (action=default)
url = 'http://10.210.201.5/api/v2/cmdb/firewall/address?action=default'
response = session.get(url=url)
print(response.text)

In [ ]:
# Cast data as a dict to read/use it.
data = json.loads(response.text)

In [ ]:
# Request - post
url = 'http://10.210.201.5/api/v2/cmdb/firewall/address'
data = {
    'name': 'address jimmy lin 10.210.201.168/32',
    'type': 'ipmask',
    'subnet': '10.210.201.168 255.255.255.255',
}
headers = {
    'X-CSRFTOKEN': session.cookies['ccsrftoken'][1:-1]
}
response = session.post(url=url, data=json.dumps(data), headers=headers)
print(response.text)

In [ ]:
# Request - put
mkey = 'address jimmy lin 10.210.201.168/32'
url = 'http://10.210.201.5/api/v2/cmdb/firewall/address/' + urllib.parse.quote(mkey, safe='')
data = {
    'name': 'address__jimmy_lin__10.210.201.168/32',
    'type': 'ipmask',
    'subnet': '10.210.201.168 255.255.255.255',
}
headers = {
    'X-CSRFTOKEN': session.cookies['ccsrftoken'][1:-1]
}
response = session.put(url=url, data=json.dumps(data), headers=headers)
print(response.text)

In [ ]:
# Request - delete
mkey = 'address jimmy lin 10.210.201.168/32'
url = 'http://10.210.201.5/api/v2/cmdb/firewall/address/' + urllib.parse.quote(mkey, safe='')
headers = {
    'X-CSRFTOKEN': session.cookies['ccsrftoken'][1:-1]
}
response = session.delete(url=url, headers=headers)
print(response.text)

In [ ]:
# Logout
url = 'http://10.210.201.5/logout'
session.post(url=url)